In [1]:
import draw

import numpy as np
from collections import defaultdict, Counter
from faker import Faker
from tabulate import tabulate
fake = Faker()

In [2]:
import importlib
importlib.reload(draw)

<module 'draw' from '/Users/cpj/Documents/code/lss/draw/draw.py'>

## Draw Simulation Methods

In [3]:
def init_players(num_players):
    """
    Create a collection of players in the form [player_id, player_name, player_ranking]
    Arguments:
        num_players: Number of players to add to the draw
    Returns:
        players: Array of player details
            (n x 3) list containing [player_id, player_name, player_ranking]
    """
    players = []
    id_to_name, id_to_rank = defaultdict(lambda: 'Bye'), defaultdict(lambda: 1600)
    for n in range(num_players):
        p_id, p_name, p_rank = np.random.randint(1000000, 9999999), fake.name(), np.random.randint(1600, 2000)
        players.append(['a' + str(p_id), p_name, p_rank])
        id_to_name['a' + str(p_id)] = p_name
        id_to_rank['a' + str(p_id)] = p_rank
    return players, id_to_name, id_to_rank

In [4]:
def init_pairings(players):
    """
    Randomly create results for the first round
    Arguments:
        players: (n x 3) numpy array containing [player_id, player_name, player_ranking]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    for n in range(0, len(players), 2):
        a, b = players[n][0], players[n+1][0]
        result = [a, b, np.random.choice([a, b, None], 1, p=(0.45, 0.45, 0.1))[0]]
        results.append(result)
    return results

In [5]:
def run_round(pairings, rnd, p_draw=0):
    """
    Randomly create results for a round
    Arguments:
        pairings: List of pairings for the round to be simulated
            (n x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    p_win = (1 - p_draw) / 2
    BYE = 0
    for pairing in pairings:
        a, b, pod = pairing[0][1], pairing[1][1], pairing[2]
        if a[:3] == 'BYE':
            result = [a,b,b]
        elif b[:3] == 'BYE':
            result = [a,b,a]
        else:
            result = [a, b, np.random.choice([a, b, None], 1, p=(p_win, p_win, p_draw))[0]]
        results.append(result + [rnd, pod])
        
    return results

## Draw Printing Methods

In [6]:
def print_standings(standings, id_to_name):
    """
    Print standings for the round in table form
    """
    table = []
    for rank, player in enumerate(standings):
        table.append([player[0], id_to_name[player[0]], player[1], rank+1, player[2], player[3], player[4]])
    print('\n', tabulate(table, headers=['ID', 'Name', 'Wins', 'Standing', 'Tiebreaks', 'Pod', 'Pod Wins']), '\n')
    

def print_pairings(pairs, id_to_name, scores):
    """
    Print pairings for the round in table form
    """
    table = []
    for table_num, pair in enumerate(pairs):
        a, b, pod = pair[0], pair[1], pair[2]
        score = scores[a[0], b[0]]
        row = [table_num+1, pod, id_to_name[a[1]], a[1], int(a[0])+1, id_to_name[b[1]], b[1], int(b[0])+1, score]
        table.append(row)
    headers = ['Table', 'Pod', 'Player 1 - Name', 'ID', 'Rank', 'Player 2 - Name', 'ID', 'Rank', 'GOF']
    print('\n', tabulate(table, headers=headers), '\n')

def print_byes(pairs_h):
    byes = []
    for pairs in pairs_h:
        for pair in pairs:
            a, b = pair[0][1], pair[1][1]
            if a[:3] == 'BYE': byes += [[b], ['BYE']]
            if b[:3] == 'BYE': byes += [[a], ['BYE']]

    byes = Counter(np.reshape(byes, -1))

    if len(byes) > 0:
        print('----- BYES -----')
        for k, v in byes.items():
            if k[:3] == 'BYE':
                print('Byes', ': ', v)
            else:
                print(k, ': ', v)

## Run a sample Tournament
Create a list of players then run through a multi-round tournament and print the results
Each round prints:
- Goodness of fit score: Score for the pairings allocated in the round. The maximum possible score is 4. If a number < 0 comes out, something has probably gone wrong
- Standings: A table of standings of the players (based only on # of wins, no accounting for draws, op wins etc)
- Pairings: A table of pairings for the following round (including player name and standing so you can check if players are being paired with others nearby)

To show all the standings and pairings at once, make sure to select the cell, click on the "Cell" menu, select -> "Current Outputs" and then select "Toggle Scrolling"

The entire history of "standings", "splits", "scores", "allocations", and "pairs" is recorded in the lists *standings_h, splits_h, scores_h, alloc_h, pairs_h*
    
Note: Early stage demo only. This does not support:
- Odd numbers of players and byes [Done]
- Allocating pods [Done]
- Iterative sampling methods to improve draw quality [Done]
- Balancing tail end pod sizes [Partly Done]
- Variable size pods [Done]
- etc

In [12]:
NUMBER_OF_PLAYERS = 32
ROUNDS  = ['swiss1', 'swiss1', 'swiss1',
           'pod1', 'pod1', 'pod1',
           'pod2', 'pod2', 'pod2',
           'swiss2', 'swiss2', 'swiss2',]
NUMBER_OF_ROUNDS = len(ROUNDS)
PROBABILITY_OF_DRAW = 0.0001
NUMBER_OF_DROPS = 0

In [13]:
# Create players
players, id_to_name, id_to_rank = init_players(NUMBER_OF_PLAYERS)

In [16]:
results = []
standings_h, splits_h, scores_h, alloc_h, pairs_h = [], [], [], [], []

for rnd in range(NUMBER_OF_ROUNDS):
    isLastRound = rnd==len(ROUNDS)-1
    standings, splits, scores, alloc, pairs = draw.create_draw(players, results, ROUNDS,
                                                               mode='test', minimiseTiebreakDist=isLastRound)
    for collection, value in zip([standings_h, splits_h, scores_h, alloc_h, pairs_h],
                                 [standings,   splits,   scores,   alloc,   pairs]):
        collection.append(value)
    n_players = len(standings)
    brackets = [n for n in range(n_players) if (standings[n,[1]] != standings[n-1,[1]]).any()] + [n_players]
    n_odd_brackets = sum(np.array(brackets) % 2)
    max_score = draw.Points.valid - 2 * n_odd_brackets * \
                (draw.Points.valid + draw.Points.overall_win_distance) / n_players
    point_freq = Counter([score for score in scores[alloc==1].astype(int)])

    print('----- Round', rnd + 1, '-----', )
    print('Goodness of Fit:', (scores * alloc).sum() / len(scores))
    print('Maximum GOF:', max_score)
    print('Frequencies: ' + ', '.join([str(int(k)) + 'pts: ' + str(int(point_freq[k] / 2)) 
                                 for k in sorted(point_freq.keys())[::-1]]))
    if NUMBER_OF_PLAYERS <= 64:
        print_standings(standings, id_to_name, )
        print_pairings(pairs, id_to_name, scores)
    result = run_round(pairs, rnd, p_draw=PROBABILITY_OF_DRAW)
    results += result
    
    for _ in range(NUMBER_OF_DROPS):
        drop_id = np.random.randint(len(players))
        del(players[drop_id])

if NUMBER_OF_PLAYERS <= 64:
    print('----- Final Standings -----')
    print_standings(draw.get_standings(players, results, ROUNDS + [ROUNDS[-1]]), id_to_name, )
    
point_freq = Counter([y for x in [s[a==1].astype(int) for s, a in zip(scores_h, alloc_h)] for y in x])
print('Overall Pairing Score Frequency: ', ', '.join([str(int(k)) + 'pts: ' + str(int(point_freq[k] / 2))
        for k in sorted(point_freq.keys())[::-1]]))
print_byes(pairs_h)

----- Round 1 -----
Goodness of Fit: 16.0
Maximum GOF: 16.0
Frequencies: 16pts: 16

 ID        Name                Wins    Standing    Tiebreaks    Pod    Pod Wins
--------  ----------------  ------  ----------  -----------  -----  ----------
a8106432  Michelle Walker        0           1            0      1           0
a1242018  Lisa Tucker            0           2            0      1           0
a9854400  Nancy Meza             0           3            0      1           0
a2649656  Diane Cook             0           4            0      1           0
a4319567  Debra Ellis            0           5            0      1           0
a8918063  Jonathan Schmidt       0           6            0      1           0
a3128439  Stephen Thomas         0           7            0      1           0
a3801234  Martin White           0           8            0      1           0
a5643249  Steven Garcia          0           9            0      1           0
a9926133  Brian Massey Jr.       0          10

----- Round 7 -----
Goodness of Fit: 13.0
Maximum GOF: 13.0
Frequencies: 16pts: 10, 8pts: 6

 ID        Name                Wins    Standing    Tiebreaks    Pod    Pod Wins
--------  ----------------  ------  ----------  -----------  -----  ----------
a8106432  Michelle Walker        6           1     1             1           0
a3538075  Jessica Evans          5           2     0.834561      1           0
a2744834  Curtis Roberts         5           3     0.824221      1           0
a6303117  Dana Peters            4           4     0.691353      1           0
a3544962  Terry Carr             4           5     0.691224      1           0
a1242018  Lisa Tucker            4           6     0.66085       1           0
a2649656  Diane Cook             4           7     0.649994      1           0
a8922558  Jennifer Stevens       4           8     0.647279      1           0
a8861251  Michael Anderson       4           9     0.691353      2           0
a7748113  Brandy King            4   

MH Iterations: 104
MH Iterations: 11
----- Round 9 -----
Goodness of Fit: 9.375
Maximum GOF: 12.5
Frequencies: 16pts: 5, 14pts: 2, 8pts: 6, 6pts: 2, -18pts: 1

 ID        Name                Wins    Standing    Tiebreaks    Pod    Pod Wins
--------  ----------------  ------  ----------  -----------  -----  ----------
a8106432  Michelle Walker        8           1     1             1           0
a3538075  Jessica Evans          6           2     0.772249      1           0
a2744834  Curtis Roberts         6           3     0.747315      1           0
a6303117  Dana Peters            6           4     0.741177      1           0
a1242018  Lisa Tucker            5           5     0.641941      1           0
a2649656  Diane Cook             5           6     0.616983      1           0
a3544962  Terry Carr             4           7     0.521062      1           0
a8922558  Jennifer Stevens       4           8     0.519032      1           0
a5906655  Donald Navarro         6           9   

MH Iterations: 99
----- Round 11 -----
Goodness of Fit: 11.0
Maximum GOF: 14.0
Frequencies: 16pts: 9, 8pts: 6, -16pts: 1

 ID        Name                Wins    Standing    Tiebreaks    Pod    Pod Wins
--------  ----------------  ------  ----------  -----------  -----  ----------
a8106432  Michelle Walker        9           1     0.922414      1           0
a2744834  Curtis Roberts         8           2     0.793509      1           0
a3538075  Jessica Evans          7           3     0.717159      1           0
a8861251  Michael Anderson       7           4     0.691373      1           0
a6553006  Sophia Montoya         7           5     0.689657      1           0
a6303117  Dana Peters            6           6     0.618636      1           0
a5906655  Donald Navarro         6           7     0.618542      1           0
a1242018  Lisa Tucker            6           8     0.59432       1           0
a6448333  Alicia Hall MD         6           9     0.59278       1           0
a8918063

#### Test Code Area

#### Test pod bracket sizing logic

In [42]:
def get_pods(players):

    mvp = 6 # Minimum viable pod
    tvp = 8 # Target viable pod

    pods = []

    while players > 0:
        if players > (mvp + tvp):
            pods += [tvp]
        elif players < mvp:
            pods += [players]
        else:
            pods += [mvp + players % mvp]
            players -= pods[-1]
            if players > 0:
                pods += [players]
        players -= pods[-1]

    positions = []
    for n, p in enumerate(pods):
        positions += [n+1] * p

    return np.array(positions)